In [16]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import math
import warnings

from IPython.display import display
from sklearn import linear_model
import statsmodels.formula.api as smf

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [17]:
df = pd.read_csv('C:\\Users\\STEVEN\\Documents\\Thinkful\\florida_crime.csv')

In [18]:
df = df.fillna(0)
print(df.loc[0,:])

City                                      Alachua
Population                                  9,338
Violent\ncrime                                 36
Murder and\nnonnegligent\nmanslaughter      0.000
Rape\n(revised\ndefinition)1                2.000
Rape\n(legacy\ndefinition)2                 0.000
Robbery                                         2
Aggravated\nassault                            32
Property\ncrime                               166
Burglary                                       40
Larceny-\ntheft                               120
Motor\nvehicle\ntheft                           6
Arson                                       1.000
Unnamed: 13                                 0.000
Name: 0, dtype: object


In [19]:
#lets create some features and clean up

df['Population'] = df['Population'].str.replace(',', '')
df = df.fillna(0)
df['Population'] = df['Population'].astype(int)

In [20]:
df['Property\ncrime'] = df['Property\ncrime'].str.replace(',', '')
df = df.fillna(0)
df['Property\ncrime'] = df['Property\ncrime'].astype(int)

df['Violent\ncrime'] = df['Violent\ncrime'].str.replace(',', '')
df = df.fillna(0)
df['Violent\ncrime'] = df['Violent\ncrime'].astype(int)

df['Larceny-\ntheft'] = df['Larceny-\ntheft'].str.replace(',', '')
df = df.fillna(0)
df['Larceny-\ntheft'] = df['Larceny-\ntheft'].astype(int)

In [21]:

df['Aggravated\nassault'] = df['Aggravated\nassault'].str.replace(',', '')
df = df.fillna(0)
df['Aggravated\nassault'] = df['Aggravated\nassault'].astype(int)



In [22]:
#add categorical aggravated assault 
df['assualt_cat'] = df['Aggravated\nassault'].apply(lambda x: 0 if x == 0 else 1)

In [23]:
df['Motor\nvehicle\ntheft'] = df['Motor\nvehicle\ntheft'].str.replace(',', '')
df = df.fillna(0)
df['Motor\nvehicle\ntheft'] = df['Motor\nvehicle\ntheft'].astype(int)

In [24]:
#Create population squared
df['Population_sq'] = df['Population'].apply(lambda x: x*x)
# Robbery
df['Robbery'] = df['Robbery'].str.replace(',', '')
df = df.fillna(0)
df['Robbery'] = df['Robbery'].astype(int)
#Creating categorical murders
df = df.rename(index=str,columns={'Murder and\nnonnegligent\nmanslaughter': 'Murder'})
df['Murder'] = df['Murder'].astype(int)
df['murder_cat'] = df['Murder'].apply(lambda x: 0 if x == 0 else 1)

In [25]:
df['Population_sqrt'] = np.sqrt(df['Population'])

df['Burglary'] = df['Burglary'].str.replace(',', '')
df = df.fillna(0)
df['Burglary'] = df['Burglary'].astype(int)
df['burglary_cat'] = df['Burglary'].apply(lambda x: 0 if x == 0 else 1)

In [26]:
df.shape

(293, 19)

In [51]:
X = df.drop(['City', 'Rape\n(revised\ndefinition)1', 'Rape\n(legacy\ndefinition)2'], axis=1)
Y = df['Violent\ncrime']
X2 = X.drop(['Violent\ncrime'], axis=1)
print(X.shape)

(293, 16)


In [28]:
df_test2 = X.sample(frac=0.5)
df_train2 = X.sample(frac=0.5)

In [45]:
#Create training and test data
trainsize = int(df.shape[0] / 2)
df_test = X.iloc[trainsize:, :].copy()
df_train = X.iloc[:trainsize, :].copy()

#Train data
Y_train = df_train['Violent\ncrime'].values.reshape(-1, 1)
X_train = df_train.loc[:, ~(df_train.columns).isin(['Violent\ncrime'])]
Y_train = df_train['Violent\ncrime']
#Test data
Y_test = df_test['Violent\ncrime'].values.reshape(-1, 1)
X_test = df_test.loc[:, ~(df_train.columns).isin(['Violent\ncrime'])]

In [46]:
print(X_train.head())
print(Y_train)

   Population  Murder  Robbery  Aggravated\nassault  Property\ncrime  \
0        9338       0        2                   32              166   
1       42040       0       37                  105             1564   
2         536       0        0                    1                3   
3        2269       0        0                    1               35   
4       45397       1       57                  109             1527   

   Burglary  Larceny-\ntheft  Motor\nvehicle\ntheft  Arson  Unnamed: 13  \
0        40              120                      6  1.000        0.000   
1       300             1180                     84  3.000        0.000   
2         3                0                      0  0.000        0.000   
3         8               27                      0  0.000        0.000   
4       231             1202                     94  6.000        0.000   

   assualt_cat  Population_sq  murder_cat  Population_sqrt  burglary_cat  
0            1       87198244           0

In [47]:
#Logistic Regression training data:


from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=10)

# Fit the model
fit = lr.fit(X_train, Y_train)

# Display.


print(fit.coef_)
print('\n R2')
print(lr.score(X_train, Y_train))

[[-1.21679759e-10 -1.15055077e-14 -1.40240215e-13 ... -8.97597672e-15
  -2.51182959e-12 -6.87314550e-14]
 [-6.00013915e-11 -4.01035714e-15 -5.68179197e-14 ... -3.15087358e-15
  -1.32550512e-12 -2.70595325e-14]
 [-1.33624144e-10 -1.06866395e-14 -1.15968430e-13 ... -8.35624165e-15
  -3.42633509e-12 -8.13291521e-14]
 ...
 [-4.87384928e-13 -2.09575845e-17 -1.27482771e-16 ... -1.10821555e-17
  -5.92503221e-15 -8.18111096e-17]
 [-6.93810078e-05 -2.50967300e-09 -2.64612200e-08 ... -1.50194427e-09
  -6.87794533e-07 -8.13174768e-09]
 [-1.59940384e-04 -6.46907051e-09 -2.03431803e-07 ... -3.13242195e-09
  -1.56851872e-06 -2.04766159e-08]]

 R2
0.0136986301369863


In [54]:
#Now to run it with our holdout comparison and cross folding.  
from sklearn.model_selection import train_test_split

X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2, Y, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(lr.fit(X_train2, Y_train2).score(X_test2, Y_test2)))
print('Testing on Sample: ' + str(lr.fit(X2, Y).score(X2, Y)))


With 20% Holdout: 0.0
Testing on Sample: 0.006825938566552901


In [53]:
#Now run as cross validation: 
from sklearn.model_selection import cross_val_score
cross_val_score(lr, X_test, Y_test, cv=2)


C:\Users\STEVEN\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
C:\Users\STEVEN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\STEVEN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0.01785714, 0.        ])

In [35]:
# now it's time for ridge regression..
from sklearn import linear_model
from sklearn import preprocessing

ridgeregr = linear_model.Ridge(alpha=5) 

# Fit the model
fit = ridgeregr.fit(X_train, Y_train)

# Display.



print('\n R2')
print(ridgeregr.score(X_train, Y_train))



 R2
0.9998633400549389


C:\Users\STEVEN\Anaconda3\lib\site-packages\scipy\linalg\basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 9.785413446166525e-24 / 1.1102230246251565e-16
  RuntimeWarning)


In [56]:
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2, Y, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(ridgeregr.fit(X_train2, Y_train2).score(X_test2, Y_test2)))
print('Testing on Sample: ' + str(ridgeregr.fit(X2, Y).score(X2, Y)))

With 20% Holdout: 0.9862211746845851
Testing on Sample: 0.9997518671681969


C:\Users\STEVEN\Anaconda3\lib\site-packages\scipy\linalg\basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 8.411880125496074e-23 / 1.1102230246251565e-16
  RuntimeWarning)
C:\Users\STEVEN\Anaconda3\lib\site-packages\scipy\linalg\basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 8.836555264940908e-24 / 1.1102230246251565e-16
  RuntimeWarning)


In [37]:
from sklearn.model_selection import cross_val_score
cross_val_score(ridgeregr, X_test, Y_test, cv=10)

C:\Users\STEVEN\Anaconda3\lib\site-packages\scipy\linalg\basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 1.994048315640466e-22 / 1.1102230246251565e-16
  RuntimeWarning)
C:\Users\STEVEN\Anaconda3\lib\site-packages\scipy\linalg\basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 9.24102294655063e-23 / 1.1102230246251565e-16
  RuntimeWarning)
C:\Users\STEVEN\Anaconda3\lib\site-packages\scipy\linalg\basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number/precision: 9.206527527107344e-23 / 1.1102230246251565e-16
  RuntimeWarning)
C:\Users\STEVEN\Anaconda3\lib\site-packages\scipy\linalg\basic.py:40: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not 

array([0.99660074, 0.99702223, 0.99892496, 0.99847739, 0.99694572,
       0.99885043, 0.99883106, 0.99827847, 0.99901453, 0.99811488])

In [38]:
#let's do lasso next

In [39]:
lass = linear_model.Lasso(alpha=1000)

# Fit the model
fit = lass.fit(X_train, Y_train)

# Display.



print('\n R2')
print(lass.score(X_train, Y_train))



 R2
0.9963893114063697


In [55]:
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2, Y, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(lass.fit(X_train2, Y_train2).score(X_test2, Y_test2)))
print('Testing on Sample: ' + str(lass.fit(X2, Y).score(X2, Y)))

With 20% Holdout: 0.9524651320735912
Testing on Sample: 0.9989715019051645


In [41]:
from sklearn.model_selection import cross_val_score
cross_val_score(lass, X_test, Y_test, cv=10)

array([0.94732074, 0.99813795, 0.97568997, 0.97567456, 0.94583006,
       0.99167317, 0.99916342, 0.99157299, 0.99859939, 0.98815752])

There were a few different interesting things that were discovered during the analysis.  

While searching through the data we included a few different features in predicting population with crime.  We had population squared and the square root of population in there, to see if they were functional.  We included for low frequency crimes a categorical feature for them.  For the features of murder, burglary, and assault.

After looking through the different models.  And testing on test and training sets.  It looks like the ridge regression model is the most efficient amongst the test and training methods.  Although it does appear that the model may be overfitting.  Second I would choose lasso because it helps prevent overfitting and may prevent a more honest representation.  